In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!echo $HF_HOME

In [3]:
import os
os.environ["HF_HOME"] = "/openbayes/home/huggingface"

## Hugging Face 是什么？

**Hugging Face** 是一个致力于 **人工智能和自然语言处理（NLP）** 的开源社区与平台，其核心是强大的 **Transformers 生态体系**。

### 🚀 Hugging Face 提供的核心内容

| 功能 | 描述 |
|------|------|
| **Model Hub** | 开放式模型库，可下载或分享预训练模型（GPT、BERT、LLaMA、Diffusion等） |
| **Transformers库** | 统一接口，便于快速加载主流模型 |
| **Datasets** | 管理和处理大规模数据集 |
| **Tokenizers** | 高性能分词器 |
| **Diffusers** | 扩散模型库（支持文生图、图生图） |
| **Spaces** | 在线部署与分享 ML 应用的平台（基于 Gradio / Streamlit） |
| **AutoTrain** | 无代码训练模型 |

### ✨ 可执行示例代码

```python
from transformers import pipeline
generator = pipeline("text-generation", model="gpt2")
print(generator("Hello"))
```

##  什么是因果语言模型？

**因果语言模型（Causal Language Model，CLM）** 是一种只能利用 **过去已出现的文本信息** 来预测 **下一个 token** 的自回归语言模型。  
其核心思想是：文本生成过程严格遵循 **时间因果顺序**，不能看到未来内容。

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 加载模型和tokenizer
# 使用AutoModelForCausalLM自动加载一个因果语言模型（causal language model）
# from_pretrained方法用于从Hugging Face Hub加载预训练模型
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",# 指定模型的名称
    device_map="auto",# 自动分配模型到可用的设备（如GPU）
    torch_dtype="auto",# 自动选择合适的PyTorch数据类型
    trust_remote_code=True,# 允许执行模型仓库中自定义的 Python 代码（某些模型结构不是标准的）；这个设置非常关键，因为有些模型不是用transformers的标准结构写的（如Qwen），而是提供了自定义类，这一参数允许从远程加载这些定义

)

# 打印模型的结构（实际上时pyTorch模型的结构）
# 可以看到模型的模块组成、层数、参数数量等信息
print(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

## 整体结构
`Qwen2ForCausalLM`是一个因果语言模型，属于Decoder-only Transformer架构，用于自回归生成任务。
结构概览：
```rust
输入 → 词嵌入 Embedding → 24层DecoderLayer → RMSNorm → lm_head → 输出 logits 
```
### 主要组件解析
1. `Embedding(151936, 896)`
     - 词表大小：151936
          - 在 NLP 模型中，文本会被切分成 tokens（词元），每个 token 都会对应一个词表中的编号（id）。词表的大小就是模型能够识别的 不同 token 的数量总和。
     - 每个token用896维向量表示
          - 模型一共支持 151,936 个可能的 token，每个 token 被映射成一个 896 维向量。
2. `ModuleList(24 x Qwen2DecoderLayer)`

     模型包含24层Transformer Decoder层，每层结构如下：
     ```css
     Input
     └── RMSNorm
          └── Self Attention
     └── RMSNorm
          └── MLP (前馈网络)
     ```
3. `Qwen2Attention`（自注意力层）
     ```scss
     (q_proj):  Linear(896 -> 896)
     (k_proj):  Linear(896 -> 128)
     (v_proj):  Linear(896 -> 128)
     (o_proj):  Linear(896 -> 896)
     ```
     说明：
     - Q、K、V 维度不同（不是常见的等维度）
     - attention head 的 key/value 维度较小（128），可节省显存并提高速度
     - 使用 RoPE 旋转位置编码 (rotary_emb)
          - 这种结构属于 Multi-Query Attention (MQA) 或 Grouped-Query Attention 的变体，即不同 head 共享 key/value，有利于推理加速。

     解释:
     - Transformer（包括 Qwen2、ChatGPT 等）在理解文本时，依靠一种叫 自注意力机制（Self Attention） 的技术。可以简单理解为模型在读一句话时，会“思考”每个词和其他词之间的关系。
     - Attention中的Q、K、V是什么？
          | 名称           | 像什么   | 作用              |
          | ------------ | ----- | --------------- |
          | **Q（Query）** | 提问题的人 | 我要关注谁？          |
          | **K（Key）**   | 关键词   | 我是不是你要找的人？      |
          | **V（Value）** | 具体信息  | 如果关注我，我能提供什么内容？ |
          
          简单比喻：
          > 你到图书馆找一本有关 AI 的书
          > - 你的需求就是 **Q**
          > - 每本书的标签是 **K**（比如：AI、历史、哲学…）
          > - K 匹配后，你真正阅读到的内容就是 **V**
          
          系统根据 Q 和 K 的匹配程度决定 **哪些词应该重点关注**，然后再去读取对应 V。
     - `(q_proj): Linear(896 -> 896)`

          这是一个线性映射层，作用是把输入的数据（词向量）转换成对应的Q、K、V

          | 名称                   | 含义                  |
          | -------------------- | ------------------- |
          | `Linear(896 -> 896)` | 输入向量维度 896，输出也为 896 |
          | `Linear(896 -> 128)` | 缩小维度到 128，提高速度、降低显存 |

          因为Q需要表达更复杂的需求，需要保持高维度，而K/V主要用于匹配，不需要太大维度。这样做可以
          - 减少计算
          - 加快生成速度
          - 降低现存消耗

     - `Multi-Query Attention (MQA)`

          传统的 Attention 每个头都有自己独立的 Q、K、V，多头结构例如：
          ```scss
          head1(Q,K,V)
          head2(Q,K,V)
          ...
          headN(Q,K,V)
          ```
          而MQA的做法是：多个头共享K、V，只保留不同的Q

          类比：
          > 一个公司多个部门（不同 heads）都要查同一个数据库（共享 K/V），不用为每个部门复制一份数据库 → 节省资源、查询更快。

     - RoPE 旋转位置编码

          因为模型本身不知道词的顺序，需要位置编码帮助它理解。

          RoPE（Rotary Position Embedding）是一种更先进的位置编码方式，让模型：
          - 更灵活理解远距离词语关系
          - 更适合长文本
4. `Qwen2MLP`（前馈网络层）
     ```yaml
     gate_proj: Linear(896 -> 4864)
     up_proj:   Linear(896 -> 4864)
     down_proj: Linear(4864 -> 896)
     (act_fn):  SiLU 激活
     ```
     
     解释：

     - MLP（多层感知机）是一个简单的神经网络结构，在每层Transformer中负责理解和提取更复杂的语义信息。

          >模型的Attention负责理解词之间的关系；
          >
          >MLP则负责更深层理解词本身的含义。

     - MLP处理一个向量（维度896），然后扩展到更大的维度（4864）,再缩回去。

          >输入：896维的token表示
          >
          >->扩张到4864维（拆分成更多特征来理解）
          >
          >->压缩回896维，将提炼后的信息返回
     - `Gated MLP`
          门控机制 → 像闸门一样控制信息流动，让模型学习“哪些信息重要，哪些要丢掉”

          优点：
          - 更聪明
          - 更省参数
          - 表达能力更强
          - 大模型训练更稳定
5. `Qwen2RMSNorm`
- RMSNorm 是一种 归一化方法，用于让训练更稳定。
- 归一化的作用：
     - 调整数据范围，避免爆炸或消失
     - 让训练更容易收敛
     - 提升效果

     | LayerNorm | RMSNorm     |
     | --------- | ----------- |
     | 同时考虑均值和方差 | 只考虑方差（平方均值） |
     | 计算更重      | 更轻，更适合大模型   |
     | 稳定性稍差     | 更稳定         |

6. `lm_head: Linear(896 → 151936)`
- 最后输出 logits，用于 softmax 预测下一个 token
     - 输入：模型内部896维表示
     - 输出：一个包含151936个数字的向量
     - 每个数字表示一个词的概率

In [8]:
# 从hugging face model hub加载与指定模型配套的分词器
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
# 返回分词器中定义的所有特殊token及其含义
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [9]:
# 构建prompt和tokenizer

prompt = "讲一个猫有关的笑话"
# 使用 Qwen 的多轮对话结构，构建一条完整的消息序列
message = [
    {"role":"system","content":"You are Qwen, create by Alibaba Cloud. You are a helpful assistant."},
    {"role":"user","content":prompt}
]

# 应用 Qwen 专用的聊天模板，将结构化的messages转换成“模型预训练时使用的原始文本格式”
text = tokenizer.apply_chat_template(
    message, # 传入对话消息列表
    tokenize = False, # 不进行分词，只返回拼接后的纯字符串。即得到的text是str，而不是token ids
    add_generation_prompt = True # 在末尾加上 assistant 的起始标记，引导模型开始“生成回答”，即告诉模型“现在轮到你说话了”
)

# 将拼接后的文本编码成模型可接受的张量(分词、映射、张量化)
model_inputs = tokenizer(
    [text],# 用列表是因为tokenizer默认支持batch，方便以后一次推理多条输入
    return_tensors="pt" # 返回PyTorch的张量格式
).to(model.device)

print(text)
print("====" * 10)
# 打印input_ids和attention_mask
## input_ids表示每个token的id
## attention_mask用于指示模型哪些token是有效的
print(model_inputs)

<|im_start|>system
You are Qwen, create by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
讲一个猫有关的笑话<|im_end|>
<|im_start|>assistant

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   1855,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,  99526,  46944, 100472,
         101063,   9370, 109959, 151645,    198, 151644,  77091,    198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


## model_inputs字段解释
- input_ids
  每个数字对应一个token，这些数字（id）来自tokenizer的词表
- attention_mask
  1表示有效，0表示填充，这里一般全是1

In [11]:
# 推理和解码

# 推理
## generate()是hugging face的自动文本生成接口，会根据输入的input_ids生成新的token序列
## 默认是贪婪解码（即每一步选择概率最高的token）

generated_ids = model.generate(
    **model_inputs, # tokenizer编码后的输入
    max_new_tokens=512 # 最多生成512个新token
)

# 截取生成的回答部分
generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)# 从生成结果中去掉原始prompt部分，仅保留新生成的token
]

# 解码：将生成的token ids转成可读的文本
response = tokenizer.batch_decode(
    generated_ids,
    skip_special_tokens=True # 去除特殊标记，如 <|im_start|>、<|im_end|> 等
)[0]# 取第一条结果（因为这里只有一条输入）

print(response)

当然，这里有一个关于猫的有趣笑话：

为什么猫咪总是在睡觉？因为他们知道“夜”就是“睡”。

这个小幽默来自中文谚语，“夜猫子”，原意是说那些在夜晚出来觅食、玩耍或出没的人。这里的“夜”字与猫的行为相似，都表示夜晚或晚上。所以，无论何时何地，如果看到一只猫在夜间活动，人们都会觉得它像是夜行性的动物，因为它们确实会感到困倦和需要休息。

这个笑话通过巧妙的比喻和对生活细节的深刻洞察，让观众能够轻松地笑出声来，同时也带有一丝智慧和趣味性。希望这个笑话能为您的聊天增添不少欢乐！


In [12]:
# 使用transformers的pipeline简化流程
from transformers import pipeline

# 1.生成pipeline
## 创建文本生成pipeline(流水线)对象
generator = pipeline(
    "text-generation", # 指定任务类型为文本生成
    model = model, # 指定要使用的预训练模型
    tokenizer = tokenizer, # 指定对应的tokenizer(分词器),用于编码输入和解码输出
    return_full_text = False, # 只返回新生成的文本部分，不包含原始提示词
    max_new_tokens = 500,# 限制最多生成的token数量（新生成，不包含提示词）
    do_sample = False # 不进行采样（即使用贪婪解码），保持稳定性输出
)

# 2.构建prompt
## pipeline会自动使用tokenizer的chat模板来处理消息
messages = [
    {'role':"user","content":"写一个和猫有关的笑话"}
]

# 3.输出并解码
output = generator(messages) # 调用pipeline完成自动编码、推理和解码
print(output[0]['generated_text'])

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


在你面前，有一只猫。它看起来很平静，但其实内心却充满了不安。
有一天，这只猫突然对你说：“你知道吗？我最近一直在想一个问题。”
“什么问题？”
“我想知道，如果一只猫想要成为一只真正的猫，它会怎么做呢？”
“这听起来像是个好问题，”你回答说，“但是，如果你问那只猫，它可能会告诉你，它想要变成一只狗，然后又变成了一个猫，然后又变成了一个狗……”
“那真是太有趣了！”猫说，“不过，我更想知道，如果一只猫真的想要变成一只真正的猫，它会怎么做呢？”
“我会一直保持它的原始状态，直到它感到满足为止。”猫回答道。“或者，我可能会让它变成一只狗，然后变成一只猫，然后变成一只狗……”


## 采样
采样是指生成文本时，从预测的词汇概率分布中随机选择下一个词，而不是总是选择概率最高的词。这是生成式模型中常用的一种非确定性解码策略，能让输出更加多样自然。
- do_sample=False时（不采样）
- 使用贪婪解码（Greedy decoding）：每一步都选择当前概率最高的词。
- 或者使用 Beam Search（束搜索）：探索多个概率高的路径。
- 结果是确定性的 —— 每次输入相同的 prompt，输出完全一样。
- do_sample=True时（启用采样）
  - 从模型给出的词汇概率分布中，按概率随机采样一个词。
  - 常配合以下参数
  - temperature：控制随机性的程度（高温→更随机）
  - top_k：仅从概率最高的k个词中采样
  - top_p(nucleus sampling)：仅从累计概率超过p的词中采样
  - 结果是非确定性的 —— 相同输入，每次输出可能不同，更具创意但也可能偏离主题。
